## **Implement the two layer network for m-samples, n-features as we discussed in class (both FP and BP) and for N layers in the hidden layer. Split the data (you can use the log. reg. data or any other one) and train your network with 80% of the data. Test your network with the test data. Report the evaluation metrics for varying number of layers in the network. Also evaluate one more activation function (ReLU/tanh) other than sigmoid function.**


In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score



In [37]:
def activation_sigmoid(z):
    return 1/(1+np.e**(-z))
def activation_tanh(z):
    return np.tanh(z)
def sigmoid_der(data):
    return data*(1-data)
def tanh_der(z):
    return (1-np.tanh(z)**2)

In [38]:
# import some data to play with
data_ls = pd.read_csv("Logistic_regression_ls.csv",index_col=False)
X = data_ls.T.iloc[:2].T.to_numpy()
y = data_ls.T.iloc[-1:].T.to_numpy()

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(np.shape(X_train),np.shape(y_train))
N = 10

(400, 2) (400, 1)


I am creating a Neural Network with 1 hidden layer, 1 output layer with m samples features, n features in hidden layer and N neurons

In [40]:
def ForwardPropogation(X_train, weight_1, weight_2, B_1, B_2, activation):
    m1, n1 = np.shape(X_train)
    N = np.shape(weight_2)[1]

    Z_1 = weight_1@X_train.T
    Z_1 = Z_1 + B_1
    A_1 = activation(Z_1)

    n2, m2 = np.shape(A_1)
    Z_2 = weight_2@A_1
    Z_2 = Z_2 + B_2
    A_2 = activation(Z_2)
    
    return {
        'A_1':A_1,
        'A_2':A_2,
        'B_1':B_1,
        'B_2':B_2,
        'weight_1':weight_1,
        'weight_2':weight_2,
        'Z_1':Z_1,
        'Z_2':Z_2
    }


In [41]:
def BackwardPropogation(X_train, y_train, data, act_der):
    m1, n1 = np.shape(X_train)

    dZ_2 = data['A_2'] - y_train.T
    dW_2 = (dZ_2 @ data['A_1'].T)/m1
    dB_2 = 1/m1*np.sum(dZ_2,axis=1,keepdims=True)

    dZ_1 = (data['weight_2'].T@dZ_2)*((act_der(data['A_1'])))
    dW_1 = 1/m1*(dZ_1@X_train)
    dB_1 = 1/m1*np.sum(dZ_1,axis=1,keepdims=True)
    
    return dW_1, dW_2, dB_1, dB_2, dZ_1, dZ_2

In [42]:
m, n = np.shape(X_train)
weight_1 = np.random.rand(N*n).reshape(N,n)
weight_2 = np.random.rand(N).reshape(1,N)
B_1 = np.ones((N,1))
B_2 = np.ones((1,1))

i = 0
while i<20000:
    data = ForwardPropogation(X_train, weight_1, weight_2, B_1, B_2, activation_sigmoid)
    data_back = BackwardPropogation(X_train,y_train,data, sigmoid_der)
    weight_1 = weight_1 - 0.1*data_back[0]
    weight_2 = weight_2 - 0.1*data_back[1]
    B_1 = B_1 - 0.1*data_back[2]
    B_2 = B_2 - 0.1*data_back[3]
    
    i+=1


In [43]:
m, n = np.shape(X_test)
print(f"Weight 1:{weight_1},\nWeight 2:{weight_2}")
predicted = ForwardPropogation(X_test, weight_1, weight_2, B_1[:,:m], B_2[:,:m], activation_sigmoid)
predicted["A_2"] = np.round(predicted["A_2"])


Weight 1:[[0.55693777 0.29441742]
 [0.86093453 0.83501481]
 [0.91619189 0.32597657]
 [0.91122385 0.7128014 ]
 [1.04658021 0.9515863 ]
 [0.87487891 0.9907018 ]
 [1.01732063 0.80070781]
 [1.31238986 0.97752661]
 [0.79571427 0.65858426]
 [0.62546626 0.83827745]],
Weight 2:[[-0.11014218 -1.18583515 -0.53275333 -0.66000713 -1.44114193 -0.86096804
   9.27432361 -1.94640534  7.0334563  -0.64424121]]


In [44]:
Accuracy = accuracy_score(y_test.T, predicted["A_2"])
Precision = precision_score(y_test.T, predicted["A_2"], average="micro")
Recall = recall_score(y_test.T, predicted["A_2"], average="micro")
F1 = f1_score(y_test.T,predicted["A_2"],average="micro")
print(f"Precision {Precision}")
print(f"Accuracy {Accuracy}")
print(f"Recall {Recall}")
print(f"F1 Score {F1}")


Precision 1.0
Accuracy 1.0
Recall 1.0
F1 Score 1.0


In [45]:
N = 5
m, n = np.shape(X_train)
weight_1 = np.random.rand(N*n).reshape(N,n)
weight_2 = np.random.rand(N).reshape(1,N)
B_1 = np.ones((N,1))
B_2 = np.ones((1,1))

data = ForwardPropogation(X_train, weight_1, weight_2, B_1, B_2, activation_tanh)
data_back = BackwardPropogation(X_train,y_train,data, tanh_der)
i = 0
while i<100000:
    
    weight_1 = weight_1 - 0.1*data_back[0]
    weight_2 = weight_2 - 0.1*data_back[1]
    B_1 = B_1 - 0.1*data_back[2]
    B_2 = B_2 - 0.1*data_back[3]
    data = ForwardPropogation(X_train, weight_1, weight_2, B_1, B_2, activation_tanh)
    data_back = BackwardPropogation(X_train,y_train,data, tanh_der)
    i+=1
m, n = np.shape(X_test)
print(f"Weight 1:{weight_1},\nWeight 2:{weight_2}")
predicted = ForwardPropogation(X_test, weight_1, weight_2, B_1[:,:m], B_2[:,:m], activation_tanh)
predicted["A_2"] = np.round(predicted["A_2"])


Weight 1:[[-0.12403953 -0.08526725]
 [ 0.74850042  0.57201649]
 [ 2.40913174  1.60361474]
 [-1.14535796 -0.89371609]
 [ 6.23902302  4.33198628]],
Weight 2:[[ 0.47272548  0.27555199  0.22366384 -2.62236329  0.72091984]]


In [46]:
Accuracy = accuracy_score(y_test.T, predicted["A_2"])
Precision = precision_score(y_test.T, predicted["A_2"], average="micro")
Recall = recall_score(y_test.T, predicted["A_2"], average="micro")
print(f"Precision {Precision}")
print(f"Accuracy {Accuracy}")
print(f"Recall {Recall}")
print(f"F1 Score {F1}")

Precision 1.0
Accuracy 1.0
Recall 1.0
F1 Score 1.0
